# piont cloud separator

In [1]:
import numpy as np
import tensorflow as tf
from pyntcloud import PyntCloud

In [2]:
def find_ranges(point_cloud):

    x_max = x_min = point_cloud[0][0]
    y_max = y_min = point_cloud[0][1]
    z_max = z_min = point_cloud[0][2]
    
    for ind, coor in enumerate(point_cloud):
        if coor[0] > x_max:
            x_max = coor[0]
        if coor[0] < x_min:
            x_min = coor[0]
        if coor[1] > y_max:
            y_max = coor[1]
        if coor[1] < y_min:
            y_min = coor[1]
        if coor[2] > z_max:
            z_max = coor[2]
        if coor[2] < z_min:
            z_min = coor[2]
            
    return ((x_min, x_max), (y_min, y_max), (z_min, z_max))


def segment_points(point_cloud, target_ranges):
    """
    Recives orig point cloud data and a tuple of boundary tuples like:
        ((x_min, x_max), (y_min, y_max), (z_min, z_max))
    """
    
    target_x_range, target_y_range, target_z_range = target_ranges[0], target_ranges[1], target_ranges[2]
    
    points = []
    for ind, coor in enumerate(point_cloud):

        if coor[0] >= target_x_range[0] and coor[0] <= target_x_range[1]:
            if coor[1] >= target_y_range[0] and coor[1] <= target_y_range[1]:
                if coor[2] >= target_z_range[0] and coor[2] <= target_z_range[1]:
                    points.append(coor)
                    
    return np.asarray(points)


def norm_point(point_cloud):
    
    ((x_min, x_max), (y_min, y_max), (z_min, z_max)) = find_ranges(point_cloud)
    
    biggest_value = np.max(np.asarray([x_max - x_min, y_max - y_min, z_max - z_min])) + 0.000000001
    
    normalized_points = []
    for ind, coor in enumerate(point_cloud):
        row = np.empty(len(coor))
        row[0] = (coor[0] - x_min)/biggest_value
        row[1] = (coor[1] - y_min)/biggest_value
        row[2] = (coor[2] - z_min)/biggest_value
        normalized_points.append(row)
        
    return np.asarray(normalized_points)

In [3]:
def traverse_point(point_cloud, cube_size=(3,3,3), overall_size=[32,32,32], stride=(1,1,1), filter_points_num=None):
    """
    It will return the target segment of point cloud.
    """
    threshold = 0
    
    if filter_points_num is not None:
        threshold = filter_points_num
    
    norm_point_cloud = norm_point(point_cloud)
    xyz_length = 1/np.asarray(overall_size)

    segmentations = []
    areas = []
    
    x_num = int((overall_size[0] - cube_size[0]) / stride[0] + 1)
    y_num = int((overall_size[1] - cube_size[1]) / stride[1] + 1)
    z_num = int((overall_size[2] - cube_size[2]) / stride[2] + 1)
    
    covers_all = (x_num == (overall_size[0] - cube_size[0]) / stride[0] + 1) and (y_num == (overall_size[1] - cube_size[1]) / stride[1] + 1) and (z_num == (overall_size[2] - cube_size[2]) / stride[2] + 1)

    for _x in range(x_num):
        for _y in range(y_num):
            for _z in range(z_num):
                target_area = ((_x*stride[0], _x*stride[0] + cube_size[0]), (_y*stride[1], _y*stride[1] + cube_size[1]), (_z*stride[2], _z*stride[2] + cube_size[2]))
    
                target_x_range = (target_area[0][0]*xyz_length[0], target_area[0][1]*xyz_length[0])
                target_y_range = (target_area[1][0]*xyz_length[1], target_area[1][1]*xyz_length[1])
                target_z_range = (target_area[2][0]*xyz_length[2], target_area[2][1]*xyz_length[2])
                
                segmented_point_cloud = segment_points(norm_point_cloud, (target_x_range, target_y_range, target_z_range))

                if len(segmented_point_cloud) > threshold:
                    
                    segmentations.append(segmented_point_cloud)
                    
                    areas.append(target_area)
                    
    # Add in the last piece of cube ()               
    if not covers_all: 
        target_area = ((x_num*stride[0], x_num*stride[0] + cube_size[0]), (y_num*stride[1], y_num*stride[1] + cube_size[1]), (z_num*stride[2], z_num*stride[2] + cube_size[2]))
    
        target_x_range = (target_area[0][0]*xyz_length[0], target_area[0][1]*xyz_length[0])
        target_y_range = (target_area[1][0]*xyz_length[1], target_area[1][1]*xyz_length[1])
        target_z_range = (target_area[2][0]*xyz_length[2], target_area[2][1]*xyz_length[2])

        segmented_point_cloud = segment_points(norm_point_cloud, (target_x_range, target_y_range, target_z_range))

        if len(segmented_point_cloud) > threshold:
            
            segmentations.append(segmented_point_cloud)   
            
            areas.append(target_area)
    
    return {'data': segmentations, 'area': areas}

In [4]:
import os
my_point_cloud = PyntCloud.from_file(os.path.join(os.getcwd(), 'ttt2.pts'), sep=" ", header=0, names=["x","y","z"])

In [5]:
segs = traverse_point(my_point_cloud.xyz, cube_size=(5,5,5), overall_size=[32,32,32], stride=(1,1,1), filter_points_num=200)

In [6]:
def voxelize3D(pts, dim=[1,1,1]):
    """
    pts: receives .pts cloud point data. 2D array, arbitary sized X,Y,Z pairs. (We will only take x,y,z into account for now)
    dim: dimensioin of output voxelized data
    
    This function will locate the grid cube and calculate the density of each cube.
    The output will be normalized values.
    """
    assert(pts.shape[1]>=3), "pts file should contain at least x,y,z coordinate"
    assert(len(dim)==3), "Please provide 3-d grid size like [32,32,32]"
    
    if len(pts) > 1:
        # move all the axis to positive area.
        minimum_val = [pts[0][0], pts[0][1], pts[0][2]]

        # find the smallest 
        for pair in pts:
            if pair[0] < minimum_val[0]:
                minimum_val[0] = pair[0]
            if pair[1] < minimum_val[1]:
                minimum_val[1] = pair[1]
            if pair[2] < minimum_val[2]:
                minimum_val[2] = pair[2]

        # move it to first quadrant 
        rectified_pts = np.empty(pts.shape)
        for index, pair in enumerate(pts):
            point = np.zeros(3)
            point[0] = pair[0] - minimum_val[0]
            point[1] = pair[1] - minimum_val[1]
            point[2] = pair[2] - minimum_val[2]
            rectified_pts[index] = point

        # biggest value in each axis 
        maximum_val = pts[0][0]

        for pair in rectified_pts:
            for val in pair:
                if val > maximum_val:
                    maximum_val = val

        # normalize all the axises to (0,1)
        normalized_pts = rectified_pts/maximum_val
    
    else:
        # in case there is just one point
        normalized_pts = pts
    
    x_grid_length = 1/dim[0]
    y_grid_length = 1/dim[1]
    z_grid_length = 1/dim[2]
    
    output = np.zeros((dim[0],dim[1],dim[2]))
    
    epsilon = 0.000000000001 # we will have at least a 1.0 value which will exceed the index of grid
    # we can use a relativly small value to escape that to fit our data
    
    max_volume_size = 0
    
    for pair in normalized_pts:
        x_loc = int(pair[0]/(x_grid_length + epsilon))
        y_loc = int(pair[1]/(y_grid_length + epsilon))
        z_loc = int(pair[2]/(z_grid_length + epsilon))
        if output[x_loc, y_loc, z_loc] is None:
            output[x_loc, y_loc, z_loc] = 1
        else:
            output[x_loc, y_loc, z_loc] += 1
        
        if output[x_loc, y_loc, z_loc] > max_volume_size:
            max_volume_size = output[x_loc, y_loc, z_loc]
    
    output = output/max_volume_size    
            
    return output

In [7]:
print(len(my_point_cloud.xyz))
print(len(segs['data'][0]))
print(len(segs['area']))
vox_segs = []
whole_vox = voxelize3D(my_point_cloud.xyz, dim=[32,32,32])
whole_vox = whole_vox.reshape(whole_vox.shape + (1,))

for idx, value in enumerate(segs['data']):
    vox = voxelize3D(value, dim=[32,32,32])
    vox_chan = np.array(vox).reshape(vox.shape + (1,))
    vox_segs.append(vox_chan)

13367
1331
1827


In [8]:
import h5py
import os
import tensorflow as tf

# Create hdf5
hdf5_path = os.path.join(os.getcwd(), 'h5dataset', 'big_shuffled_sets.h5')
hdf5_file = h5py.File(hdf5_path, mode='r')

b_data = hdf5_file.get('voxels')
b_labels = hdf5_file.get('labels')
b_label_ref = hdf5_file.get('label_ref')

In [9]:
# one hot indexes

def predict(voxels, output_format='weights', device_name='cpu:0'):
    """
    Output_format can be 'weights' or 'probs'.
    
    'weights' will output the weight straight away which can be treated as confident.
    
    'probs' will output the corresponding probabilities for each class.
    """
    
    import os
    model_path = os.path.join(os.getcwd(), 'trained_model', 'model-2')

    config = tf.ConfigProto(allow_soft_placement = True)
    with tf.Session(graph=tf.Graph(), config=config) as sess:
        with tf.device(device_name):

            saver = tf.train.import_meta_graph(model_path + ".meta")
            saver.restore(sess, model_path)

            graph = tf.get_default_graph()
            x_input = graph.get_tensor_by_name('inputs/x_input:0')
            y_input = graph.get_tensor_by_name('inputs/y_input:0')
            pred = graph.get_collection('logits')
            accuracy = graph.get_tensor_by_name('acc:0')
            
            res = []
            
            if output_format is 'weights':
                for _, val in enumerate(voxels):
                    y = sess.run(pred, feed_dict={x_input: [val]})
                    res.append(np.asarray(y[0]))
                return res
            
            elif output_format is 'probs':
                for _, val in enumerate(voxels):
                    y = sess.run(pred, feed_dict={x_input: [val]})
                    res.append(tf.nn.softmax(np.asarray(y[0])).eval())
                return res

In [12]:
predictions = predict(vox_segs, output_format='weights', device_name='cpu:0')

INFO:tensorflow:Restoring parameters from /Users/shijian/git/3D-CNN/concept/3d_pointcloud/trained_model/model-2


In [10]:
def count_prob(probs, cube_labels, threshold=0.7):
    li = dict()
    cubes = dict()
    for _, val in enumerate(probs):
        _max = np.argmax(val)
        if val[0][_max] > threshold:
            if _max not in li:
                li.update({_max:1})
                m_li = list()
                m_li.append(cube_labels[_])
                cubes.update({_max: m_li})
            else:
                li.update({_max: li[_max]+1})
                m_li = cubes[_max]
                m_li.append(cube_labels[_])
                cubes.update({_max: m_li})
    return li, cubes

def count_confidence(confidence, cube_labels, threshold=3):
    li = dict()
    cubes = dict()
    for _, val in enumerate(confidence):
        _max = np.argmax(val)
        if val[0][_max] > threshold:
            if _max not in li:
                li.update({_max:1})
                m_li = list()
                m_li.append(cube_labels[_])
                cubes.update({_max: m_li})
            else:
                li.update({_max: li[_max]+1})
                m_li = cubes[_max]
                m_li.append(cube_labels[_])
                cubes.update({_max: m_li})
    return li, cubes

In [11]:
def to_density(occ_list, upper=[32,32,32]):
    res = np.zeros((upper[0], upper[1], upper[2]))/10
    for occ_row in occ_list:
        for _x in range(occ_row[0][1] - occ_row[0][0]):
            for _y in range(occ_row[1][1] - occ_row[1][0]):
                for _z in range(occ_row[2][1] - occ_row[2][0]):
                    res[occ_row[0][0]+_x][occ_row[1][0]+_y][occ_row[2][0]+_z] = 1
    return res

def to_union(occ_list):
    res = [[occ_list[0][0][0], occ_list[0][0][1]], [occ_list[0][1][0], occ_list[0][1][1]], [occ_list[0][2][0], occ_list[0][2][1]]]
    for occ_row in occ_list:
        if res[0][0] > occ_row[0][0]:
            res[0][0] = occ_row[0][0]
        if res[0][1] < occ_row[0][1]:
            res[0][1] = occ_row[0][1]
            
        if res[1][0] > occ_row[1][0]:
            res[1][0] = occ_row[1][0]
        if res[1][1] < occ_row[1][1]:
            res[1][1] = occ_row[1][1]
            
        if res[2][0] > occ_row[2][0]:
            res[2][0] = occ_row[2][0]
        if res[2][1] < occ_row[2][1]:
            res[2][1] = occ_row[2][1]
        
    return res

In [13]:
argmax, occ = count_confidence(predictions, segs['area'], threshold=4)
print(argmax)
print(to_union(occ[4]))
print(b_label_ref.value)

{1: 60, 2: 91, 4: 133}
[[1, 13], [4, 15], [6, 17]]
[b'car' b'lamp' b'table' b'airplane' b'chair']


In [578]:
target = occ[1]
for idx, v in enumerate(segs['area']):
    if v in target:
        print(idx, len(segs['data'][idx]))

1 1423


In [401]:
# file = open(os.path.join(os.getcwd(),"occ5out32.temp"), "w") 
# file.write(str(segs))
# file.close()

In [10]:
def write_to_file(xyz, filename='xyz'):
    import os
    file = open(os.path.join(os.getcwd(), filename + ".pts"), "w") 
    
    for point in xyz:
        st = ""
        for item in point:
            st += str(item) + " "
        file.write(st.strip() + "\n")

    file.close() 
    
# write_to_file(segmented_point_cloud)

In [22]:
# %load ./voxel_grid_plot.py
import os
import shutil

import numpy as np
import pandas as pd

from IPython.display import IFrame
from matplotlib import pyplot as plt
from pyntcloud import PyntCloud

def plot_voxel_points(voxelgrid,
                   point_cloud,
                   output_name=None,
                   cmap="Oranges",
                   axis=True,
                   width=800,
                   height=600):

    # For Voxel Grid
    scaled_shape = voxelgrid.shape

    vector = voxelgrid
    points = np.argwhere(vector)

    s_m = plt.cm.ScalarMappable(cmap=cmap)
    rgb = s_m.to_rgba(vector.reshape(-1)[vector.reshape(-1) > 0])

    camera_position = points.max(0) + abs(points.max(0))

    look = points.mean(0)

    if axis:
        axis_size = points.ptp() * 1.5
    else:
        axis_size = 0
    
    # For point cloud
    filename = 'pyntcloud_plot'
    
    path = os.path.join(os.getcwd(), filename + ".pts")
    # Fit point cloud into Voxel Grid
    (x_min, x_max), (y_min, y_max), (z_min, z_max) = find_ranges(point_cloud)
    pts_camera_position = (point_cloud.max(0) + abs(point_cloud.max(0))).tolist()
    new_point_cloud = np.empty(point_cloud.shape)
    for idx, row in enumerate(point_cloud):
        _r = np.empty(row.shape)
        _r[0] = (row[0] - x_min)/(x_max - x_min)*scaled_shape[0]
        _r[1] = (row[1] - y_min)/(y_max - y_min)*scaled_shape[1]
        _r[2] = (row[2] - z_min)/(z_max - z_min)*scaled_shape[2]
        new_point_cloud[idx] = _r
    point_cloud = new_point_cloud
    
    # Orange by default
    colors = np.repeat([[255, 125, 0]], point_cloud.shape[0], axis=0)
    colors = colors.astype(np.uint8)
    points_df = pd.DataFrame(point_cloud, columns=["x", "y", "z"])
    for n, i in enumerate(["red", "green", "blue"]):
        points_df[i] = colors[:, n]
    cloud = PyntCloud(points_df)
    
    ply_gen = cloud.to_file("{}.ply".format(filename), also_save=["mesh"])
    
#     look_at = cloud.xyz.mean(0).tolist()
    

    placeholders = {}

    placeholders["POINTS_X_PLACEHOLDER"] = points[:, 0].tolist()
    placeholders["POINTS_Y_PLACEHOLDER"] = points[:, 1].tolist()
    placeholders["POINTS_Z_PLACEHOLDER"] = points[:, 2].tolist()

    placeholders["R_PLACEHOLDER"] = rgb[:, 0].tolist()
    placeholders["G_PLACEHOLDER"] = rgb[:, 1].tolist()
    placeholders["B_PLACEHOLDER"] = rgb[:, 2].tolist()

    placeholders["S_x_PLACEHOLDER"] = 1
    placeholders["S_y_PLACEHOLDER"] = 1
    placeholders["S_z_PLACEHOLDER"] = 1

    placeholders["CAMERA_X_PLACEHOLDER"] = camera_position[0]
    placeholders["CAMERA_Y_PLACEHOLDER"] = camera_position[1]
    placeholders["CAMERA_Z_PLACEHOLDER"] = camera_position[2]

    placeholders["LOOK_X_PLACEHOLDER"] = look[0]
    placeholders["LOOK_Y_PLACEHOLDER"] = look[1]
    placeholders["LOOK_Z_PLACEHOLDER"] = look[2]

    placeholders["AXIS_SIZE_PLACEHOLDER"] = axis_size

    placeholders["N_VOXELS_PLACEHOLDER"] = sum(vector.reshape(-1) > 0)
    
    placeholders["FILENAME_PLACEHOLDER"] = "\"" + filename + "\""
    placeholders["POINT_SIZE_PLACEHOLDER"] = 0.3

    
    if output_name is None:
        output_name = "plotVG.html"
    
    BASE_PATH = os.getcwd()
    src = "{}/{}".format(BASE_PATH, "obj_detection_plot.html")
    dst = "{}/{}".format(os.getcwd(), output_name)
    point_size=0.001
    
    with open(src, "r") as inp, open(dst, "w") as out:
        for line in inp:
            for key, val in placeholders.items():
                if key in line:
                    line = line.replace(key, str(val))
            out.write(line)

    return IFrame(output_name, width=width, height=height)

In [780]:
print(b_label_ref.value)
x = np.zeros((32,32,32))
x[0:10, 0:15, 5:10] = 1
plot_voxel_points(x, my_point_cloud.xyz)
# plot_voxel_points(to_density(occ[1], upper=[32,32,32]), my_point_cloud.xyz)
# my_point_cloud.plot(point_size=0.01)

[b'car' b'lamp' b'table' b'airplane' b'chair']


In conclusion, we need to train our CNN with partial data at least for a better performance, since it will always treat partial data as a full data, wanted to give some predictions even it hasn't seen that data before.

Next, I will crop some corner, side point to train the neural net to identify is it a valid object or not.

Meanwhile, below is the implementation for unsupervised region proposal to avoid more training.

# Region Proposal
### DBSCAN

In [20]:
# https://github.com/NLeSC/PattyAnalytics/blob/master/patty/segmentation/dbscan.py

from sklearn.cluster import dbscan
def dbscan_labels(pointcloud, epsilon, minpoints, rgb_weight=0,
                  algorithm='ball_tree'):
    '''
    Find an array of point-labels of clusters found by the DBSCAN algorithm.
    Parameters
    ----------
    pointcloud : pcl.PointCloud
        Input pointcloud.
    epsilon : float
        Neighborhood radius for DBSCAN.
    minpoints : integer
        Minimum neighborhood density for DBSCAN.
    rgb_weight : float, optional
        If non-zero, cluster on color information as well as location;
        specifies the relative weight of the RGB components to spatial
        coordinates in distance computations.
        (RGB values have wildly different scales than spatial coordinates.)
    Returns
    -------
    labels : Sequence
        A sequence of labels per point. Label -1 indicates a point does not
        belong to any cluster, other labels indicate the cluster number a
        point belongs to.
    '''

    if rgb_weight > 0:
        X = pointcloud.to_array()
        X[:, 3:] *= rgb_weight
    else:
        X = pointcloud

    _, labels = dbscan(X, eps=epsilon, min_samples=minpoints,
                       algorithm=algorithm)
    return np.asarray(labels)

In [38]:
def mean_shift_labels(pointcloud, bandwidth=None, seeds=None, bin_seeding=False,
               min_bin_freq=1, cluster_all=True, max_iter=300,
               n_jobs=1):
    '''
    Find an array of point-labels of clusters found by the DBSCAN algorithm.
    Parameters
    ----------
    X : array-like, shape=[n_samples, n_features]
        Input data.
    bandwidth : float, optional
        Kernel bandwidth.
        If bandwidth is not given, it is determined using a heuristic based on
        the median of all pairwise distances. This will take quadratic time in
        the number of samples. The sklearn.cluster.estimate_bandwidth function
        can be used to do this more efficiently.
    seeds : array-like, shape=[n_seeds, n_features] or None
        Point used as initial kernel locations. If None and bin_seeding=False,
        each data point is used as a seed. If None and bin_seeding=True,
        see bin_seeding.
    bin_seeding : boolean, default=False
        If true, initial kernel locations are not locations of all
        points, but rather the location of the discretized version of
        points, where points are binned onto a grid whose coarseness
        corresponds to the bandwidth. Setting this option to True will speed
        up the algorithm because fewer seeds will be initialized.
        Ignored if seeds argument is not None.
    min_bin_freq : int, default=1
       To speed up the algorithm, accept only those bins with at least
       min_bin_freq points as seeds.
    cluster_all : boolean, default True
        If true, then all points are clustered, even those orphans that are
        not within any kernel. Orphans are assigned to the nearest kernel.
        If false, then orphans are given cluster label -1.
    max_iter : int, default 300
        Maximum number of iterations, per seed point before the clustering
        operation terminates (for that seed point), if has not converged yet.
    n_jobs : int
        The number of jobs to use for the computation. This works by computing
        each of the n_init runs in parallel.
    Returns
    -------
    cluster_centers : array, shape=[n_clusters, n_features]
        Coordinates of cluster centers.
    labels : array, shape=[n_samples]
        Cluster labels for each point.
    '''

    _, labels = mean_shift(pointcloud, bandwidth=bandwidth, seeds=seeds, bin_seeding=bin_seeding,
                            min_bin_freq=min_bin_freq, cluster_all=cluster_all, max_iter=max_iter,
                            n_jobs=n_jobs)
    return labels

In [18]:
from sklearn.cluster import SpectralClustering
clu = SpectralClustering()
spe = clu.fit(c)
spe.labels_
res = find_cluster_points(c, spe.labels_)

In [39]:
normalized_cloud = norm_point(my_point_cloud.xyz)
lab = mean_shift_labels(normalized_cloud, max_iter=3000, n_jobs=4)

In [40]:
res = find_cluster_points(normalized_cloud, lab)

In [54]:
from sklearn.cluster.k_means_ import k_means
# need exact number of clusters
la = k_means(normalized_cloud, 6, init='k-means++', precompute_distances='auto',
            n_init=10, max_iter=300, verbose=False,
            tol=1e-4, random_state=None, copy_x=True, n_jobs=1,
            algorithm="auto", return_n_iter=False)
res = find_cluster_points(normalized_cloud, la[1])

In [17]:
def find_cluster_points(point_cloud, labels):
    res = dict()
    for idx, val in enumerate(labels):
        if val not in res:
            li = list()
            li.append(point_cloud[idx])
            res.update({val: li})
        else:
            li = res[val]
            li.append(point_cloud[idx])
            res.update( {val: li} )
    
    renew_dict = dict()
    for idx, key in enumerate(res):
        renew_dict.update({key: np.asarray(res[key])})
    
    return renew_dict


def localize_region(point_cloud, target_cloud, grid_size=[32,32,32]):
    
    ((x_min, x_max), (y_min, y_max), (z_min, z_max)) = find_ranges(point_cloud)
    ((x_target_min, x_target_max), (y_target_min, y_target_max), (z_target_min, z_target_max)) = find_ranges(target_cloud)
    
    x_length = (x_max - x_min)/32
    y_length = (y_max - y_min)/32
    z_length = (z_max - z_min)/32
    
    x_region_min = int(x_target_min/x_length)
    x_region_max = int(x_target_max/x_length + 2)
    
    y_region_min = int(y_target_min/y_length)
    y_region_max = int(y_target_max/y_length + 2)
    
    z_region_min = int(z_target_min/z_length)
    z_region_max = int(z_target_max/z_length + 2)
    
    return ((x_region_min, x_region_max), (y_region_min, y_region_max), (z_region_min, z_region_max))
    

In [23]:
normalized_cloud = norm_point(my_point_cloud.xyz)
cluster_labels = dbscan_labels(normalized_cloud, 0.02, 10, algorithm='ball_tree')
res = find_cluster_points(normalized_cloud, cluster_labels)

In [32]:
points_df = pd.DataFrame(res[3], columns=["x", "y", "z"])
PyntCloud(points_df).plot(point_size=0.005)

In [24]:
cluster_labels

array([0, 0, 0, ..., 4, 4, 4])

In [25]:
my_point_cloud = PyntCloud.from_file(os.path.join(os.getcwd(), 'ttt3.pts'), sep=" ", header=0, names=["x","y","z"])
region = localize_region(normalized_cloud, res[0])
den = np.zeros((32,32,32))
den[region[0][0]:region[0][1], region[1][0]:region[1][1], region[2][0]:region[2][1]] = 1
plot_voxel_points(den, my_point_cloud.xyz)

array([[-0.16703001, -0.25178   ,  0.39102   ],
       [-0.08656   , -0.30826   ,  0.2685    ],
       [-0.07608   , -0.29803   ,  0.28215   ],
       ..., 
       [ 0.00245   , -0.25911   ,  0.30232   ],
       [-0.09092   , -0.30221   ,  0.2471    ],
       [-0.12065   , -0.27172   ,  0.26181999]])

In [746]:
voxels = []
for idx, key in enumerate(res):
    if key != -1:
        vox = voxelize3D(res[key], dim=[32,32,32])
        vox_chan = np.array(vox).reshape(vox.shape + (1,))
        voxels.append(vox_chan)

In [753]:
predictions = predict(voxels, output_format='probs', device_name='cpu:0')

INFO:tensorflow:Restoring parameters from /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-2


INFO:tensorflow:Restoring parameters from /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-2


In [755]:

print(b_label_ref.value)
predictions

[b'car' b'lamp' b'table' b'airplane' b'chair']


[array([[ 0.00343599,  0.04454208,  0.00514553,  0.89686203,  0.05001442]], dtype=float32),
 array([[  3.52182433e-05,   8.41522455e-01,   9.86838937e-02,
           1.22957979e-04,   5.96354268e-02]], dtype=float32),
 array([[ 0.02688141,  0.7284373 ,  0.13400495,  0.00968067,  0.10099573]], dtype=float32),
 array([[  1.96535271e-11,   2.92222392e-11,   9.63168191e-07,
           6.79611640e-13,   9.99999046e-01]], dtype=float32),
 array([[  1.08488756e-07,   9.49013156e-06,   9.46136415e-01,
           7.13847754e-08,   5.38539365e-02]], dtype=float32)]

# produce multi data

In [411]:
my_point_cloud_a = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'airplane', 'points', '1a74b169a76e651ebc0909d98a1ff2b4.pts'), sep=" ", header=0, names=["x","y","z"])
my_point_cloud_b = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'lamp', 'points', '1a9c1cbf1ca9ca24274623f5a5d0bcdc.pts'), sep=" ", header=0, names=["x","y","z"])
my_point_cloud_c = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'car', 'points', '1a0c91c02ef35fbe68f60a737d94994a.pts'), sep=" ", header=0, names=["x","y","z"])
my_point_cloud_d = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'chair', 'points', '1ab8a3b55c14a7b27eaeab1f0c9120b7.pts'), sep=" ", header=0, names=["x","y","z"])
my_point_cloud_e = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'table', 'points', '1a1fb603583ce36fc3bd24f986301745.pts'), sep=" ", header=0, names=["x","y","z"])

print(type(my_point_cloud_a.xyz))
c = np.concatenate((my_point_cloud_a.xyz, my_point_cloud_b.xyz + 0.5, my_point_cloud_c.xyz + 1, my_point_cloud_d.xyz - 0.5, my_point_cloud_e.xyz - 1))
print(c.shape)

write_to_file(c, filename="ttt2")

<class 'numpy.ndarray'>
(13368, 3)


In [412]:
PyntCloud.from_file(os.path.join(os.getcwd(), 'ttt2.pts'), sep=" ", header=0, names=["x","y","z"]).plot(point_size=0.01)

In [13]:
import pandas as pd
import os
my_point_cloud_b = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'lamp', 'points', '1a9c1cbf1ca9ca24274623f5a5d0bcdc.pts'), sep=" ", header=0, names=["x","y","z"])
my_point_cloud_e = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'table', 'points', '1a1fb603583ce36fc3bd24f986301745.pts'), sep=" ", header=0, names=["x","y","z"])

c = np.concatenate((my_point_cloud_b.xyz*0.35, my_point_cloud_e.xyz + np.asarray([0,-0.3,0])))

points_df = pd.DataFrame(c, columns=["x", "y", "z"])
PyntCloud(points_df).plot()
# write_to_file(c, filename="table_lamp_sample")

# Testing

In [20]:
def find_ranges(point_cloud):

    x_max = x_min = point_cloud[0][0]
    y_max = y_min = point_cloud[0][1]
    z_max = z_min = point_cloud[0][2]
    
    for ind, coor in enumerate(point_cloud):
        if coor[0] > x_max:
            x_max = coor[0]
        if coor[0] < x_min:
            x_min = coor[0]
        if coor[1] > y_max:
            y_max = coor[1]
        if coor[1] < y_min:
            y_min = coor[1]
        if coor[2] > z_max:
            z_max = coor[2]
        if coor[2] < z_min:
            z_min = coor[2]
            
    return ((x_min, x_max), (y_min, y_max), (z_min, z_max))


def find_cube_length(t_range, overall_size):
    length = (t_range[1] - t_range[0])/overall_size
    return length

def get_all_length(point_cloud, overall_size, xyz_ranges):
    
    x_length = find_cube_length(xyz_ranges[0], overall_size[0])
    y_length = find_cube_length(xyz_ranges[1], overall_size[1])
    z_length = find_cube_length(xyz_ranges[2], overall_size[2])
    
    return (x_length, y_length, z_length)
    
def find_target_area_range(point_cloud, target_area, xyz_length, xyz_ranges):
    
    target_x_range_max = xyz_ranges[0][1] - target_area[0][0]*xyz_length[0]
    target_x_range_min = xyz_ranges[0][1] - target_area[0][1]*xyz_length[0]
    target_y_range_max = xyz_ranges[1][1] - target_area[1][0]*xyz_length[1]
    target_y_range_min = xyz_ranges[1][1] - target_area[1][1]*xyz_length[1]
    target_z_range_max = xyz_ranges[2][1] - target_area[2][0]*xyz_length[2]
    target_z_range_min = xyz_ranges[2][1] - target_area[2][1]*xyz_length[2]
    
    return (target_x_range_min, target_x_range_max), (target_y_range_min, target_y_range_max), (target_z_range_min, target_z_range_max)


def segment_points(point_cloud, target_ranges):
    """
    Recives orig point cloud data and a tuple of boundary tuples like:
        ((x_min, x_max), (y_min, y_max), (z_min, z_max))
    """
    
    target_x_range, target_y_range, target_z_range = target_ranges[0], target_ranges[1], target_ranges[2]
    
    points = []
    for ind, coor in enumerate(point_cloud):
        if coor[0] >= target_x_range[0] and coor[0] <= target_x_range[1]:
            if coor[1] >= target_y_range[0] and coor[1] <= target_y_range[1]:
                if coor[2] >= target_z_range[0] and coor[2] <= target_z_range[1]:
                    points.append(coor)
                    
    return np.asarray(points)

In [ ]:
def diagonal_cubes(point_cloud, overall_size=(16,16,16)):
    
    xyz_range = find_ranges(point_cloud)
    xyz_length = get_all_length(point_cloud, overall_size, xyz_range)
    
    